In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df0=pd.read_csv(r'X:\FACT_ORDER8.csv',encoding='windows-1252')
df0 = df0[['ORDER_ID','CLIENT_ID','ORDERED','ORDER_START_LAG_DAYS','ORDER_DURATION_DAYS']]
df0 = df0.groupby(by = ['CLIENT_ID']).agg({'ORDER_ID' : 'nunique', 'ORDERED' : 'sum' , 
                                                               'ORDER_START_LAG_DAYS' : 'mean' , 'ORDER_DURATION_DAYS' : 'mean'}).reset_index()
df0.columns = ['CLIENT_ID','ORDERS','MEN_NEEDED','AVERAGE_START_LAG','AVERAGE_DURATION']
df0.head()


C:\Users\anthony.macko\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CLIENT_ID,ORDERS,MEN_NEEDED,AVERAGE_START_LAG,AVERAGE_DURATION
0,164592,1,2,3.0,0.0
1,125984,1,2,2.0,-1.0
2,154383,1,1,0.0,1.0
3,166967,1,1,3.0,-1.0
4,FO133979,1,1,1.0,-1.0


In [3]:
df1=pd.read_csv(r'X:\FACT_PRODUCTION2.csv',encoding='windows_1252')
df1 = df1[['CLIENT_ID','INVOICE_HOURS','INVOICE_GP','COST_OF_SALES','SALES']]

C:\Users\anthony.macko\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (0,16,17,18,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df1 = df1.groupby(by = ['CLIENT_ID']).agg({'INVOICE_HOURS' : 'sum', 'INVOICE_GP' : 'sum' , 'COST_OF_SALES' : 'sum' , 'SALES' : 'sum'}).reset_index()
df1.columns = ['CLIENT_ID','HOURS','GP','BURDEN','SALES']
df1['GP_DOLLAR'] = df1['GP'] / df1['HOURS']
df1.drop('HOURS', axis=1, inplace=True)
df1.drop('GP', axis=1, inplace=True)
df1.drop('BURDEN', axis=1, inplace=True)
df1.drop('SALES', axis=1, inplace=True)
df1.head()

,CLIENT_ID,GP_DOLLAR
0,0,NaN
1,100005,6.128958
2,100016,11.032391
3,100034,11.014957
4,100035,10.984450


In [5]:
df4=pd.read_csv(r'X:\DIM_CLIENT.csv',encoding='windows-1252')
df4 = df4[['CLIENT_ID','CLIENT_TSI_FLAG','CLIENT_CLC_LEGACY_FLAG','CLIENT_VINTAGE','QUADRANT','ABC_MEMBER']]
df4['CLIENT_TSI_FLAG'] = np.where(df4['CLIENT_TSI_FLAG'] == "N",0,1)
df4['CLIENT_CLC_LEGACY_FLAG'] = np.where(df4['CLIENT_CLC_LEGACY_FLAG'] == "N",0,1)
df4.head()

C:\Users\anthony.macko\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CLIENT_ID,CLIENT_TSI_FLAG,CLIENT_CLC_LEGACY_FLAG,CLIENT_VINTAGE,QUADRANT,ABC_MEMBER
0,AAWBUI,0,0,1999,0,0
1,RLMCON,0,0,2002,0,0
2,DRCCON,0,0,1996,0,0
3,ACCCON,0,0,2001,0,0
4,ABMCON,0,0,2002,0,0


In [6]:
df6 = df0.merge(df4,on=["CLIENT_ID"], how = "left")
df6 = df6.merge(df1, on =['CLIENT_ID'], how = "left" )
df6.drop('CLIENT_ID', axis=1, inplace=True)
df6= df6[(df6.GP_DOLLAR > 1) & (df6.GP_DOLLAR < 20)]
df6= df6[(df6.QUADRANT != 0)] 
df6=df6.dropna()
df6['PROFIT_LEVEL'] = pd.qcut(df6['GP_DOLLAR'], 2, labels=False)
df6.drop('GP_DOLLAR', axis=1, inplace=True)
df6.head()

,ORDERS,MEN_NEEDED,AVERAGE_START_LAG,AVERAGE_DURATION,CLIENT_TSI_FLAG,CLIENT_CLC_LEGACY_FLAG,CLIENT_VINTAGE,QUADRANT,ABC_MEMBER,PROFIT_LEVEL
7,1,6,12.000000,10.00,0.0,0.0,2004.0,4.0,0.0,0
8,1,15,64.000000,0.00,0.0,0.0,2004.0,1.0,0.0,1
9,3,4,6.666667,15.00,0.0,0.0,2004.0,1.0,0.0,1
10,4,27,114.750000,0.00,0.0,0.0,2004.0,3.0,0.0,1
11,25,31,6.720000,39.04,0.0,0.0,2004.0,4.0,0.0,0


In [7]:
df6.to_csv(r'X:\ML MODELS\ORDER_PRIORITY.csv', sep=',', encoding='utf-8', index = False)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import statsmodels.discrete.discrete_model as sm

In [9]:
X = df6.drop(["PROFIT_LEVEL"], axis=1)
y = df6["PROFIT_LEVEL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
model_features = list(X)

coef_dict = {}
for coef, feat in zip(model.coef_,'COEF'):
    coef_dict[feat] = coef
    



C:\Users\anthony.macko\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
model.coef_

array([[-0.00205972,  0.0030608 ,  0.01942065,  0.00384266,  0.38793522,
        -0.26998766,  0.00223186, -1.63706783,  0.41016871]])

In [11]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      2963
           1       0.85      0.81      0.83      2924

   micro avg       0.84      0.84      0.84      5887
   macro avg       0.84      0.84      0.84      5887
weighted avg       0.84      0.84      0.84      5887

